In [1]:
import sys
sys.path.append('../../30_data_tools/')

In [2]:
# Set up CUDA in OS
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# Import libabries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.transforms as T
from torchvision import datasets, models, transforms
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [3]:
from pathlib import Path
from helper import load_dotenv

In [4]:
dotenv = load_dotenv()

In [5]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [7]:
# Find out if a GPU is available
use_cuda = torch.cuda.is_available()
use_cuda

False

In [8]:
DATASET_DIR = Path('./dataset/')

In [9]:
available_datasets = {}
for entry in DATASET_DIR.iterdir():
    if entry.is_dir() and entry.name.startswith('.') == False:
        available_datasets[entry.name] = {
            'path' : entry
        }

In [10]:
# Create transform function
transforms_data = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]) # normalization
])

In [11]:
for key in available_datasets:
    available_datasets[key]['dataset'] = datasets.ImageFolder(available_datasets[key]['path'], transforms_data)
    available_datasets[key]['dataloader'] = torch.utils.data.DataLoader(available_datasets[key]['dataset'], batch_size=12, shuffle=True, num_workers=0)

In [12]:
for key in available_datasets:
    print(f'{ key } size', len(available_datasets[key]['dataset']))

print('class names', available_datasets[list(available_datasets.keys())[0]]['dataset'].classes)

real_val size 3916
test size 1276
train size 5051
val size 574
class names ['moire', 'no_moire']


# get model

In [13]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
num_features = model.fc.in_features 
print('Number of features from pre-trained model', num_features)

Number of features from pre-trained model 512


In [15]:
num_features

512

In [16]:
# Add a fully-connected layer for classification
model.fc = nn.Sequential(
    nn.Linear(num_features, 2),
    nn.Sigmoid()
)
model = model.to(device)

# Train

In [17]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [18]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [19]:
import time

In [22]:
#### Train model
train_loss=[]
train_accuary=[]
test_loss=[]
test_accuary=[]

num_epochs = 10   #(set no of epochs)
start_time = time.time() #(for showing time)
# Start loop
for epoch in range(num_epochs): #(loop for every epoch)
    print("Epoch {} running".format(epoch)) #(printing message)
    """ Training Phase """
    model.train()    #(training model)
    running_loss = 0.   #(set loss 0)
    running_corrects = 0 
    # load a batch data of images
    for i, (inputs, labels) in enumerate(available_datasets['train']['dataloader']):
        inputs = inputs.to(device)
        labels = labels.to(device) 
        # forward inputs and get output
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        # get loss value and update the network weights
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data).item()
    epoch_loss = running_loss / len(available_datasets['train']['dataset'])
    epoch_acc = running_corrects / len(available_datasets['train']['dataset']) * 100.
    # Append result
    train_loss.append(epoch_loss)
    train_accuary.append(epoch_acc)
    # Print progress
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch+1, epoch_loss, epoch_acc, time.time() -start_time))
    """ Testing Phase """
    model.eval()
    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0
        for inputs, labels in available_datasets['test']['dataloader']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            running_corrects += torch.sum(preds == labels.data).item()
        epoch_loss = running_loss / len(available_datasets['test']['dataset'])
        epoch_acc = running_corrects / len(available_datasets['test']['dataset']) * 100.
        # Append result
        test_loss.append(epoch_loss)
        test_accuary.append(epoch_acc)
        # Print progress
        print('[Test #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch+1, epoch_loss, epoch_acc, time.time()- start_time))

Epoch 0 running
[Train #1] Loss: 0.0390 Acc: 84.4981% Time: 163.2149s
[Test #1] Loss: 0.0376 Acc: 87.3824% Time: 179.2125s
Epoch 1 running
[Train #2] Loss: 0.0383 Acc: 85.2702% Time: 337.4601s
[Test #2] Loss: 0.0391 Acc: 85.3448% Time: 353.3042s
Epoch 2 running
[Train #3] Loss: 0.0377 Acc: 86.4977% Time: 509.7642s
[Test #3] Loss: 0.0370 Acc: 88.5580% Time: 525.5239s
Epoch 3 running
[Train #4] Loss: 0.0370 Acc: 87.1907% Time: 679.7142s
[Test #4] Loss: 0.0371 Acc: 88.0094% Time: 695.6976s
Epoch 4 running
[Train #5] Loss: 0.0366 Acc: 87.8638% Time: 851.3272s
[Test #5] Loss: 0.0369 Acc: 87.9310% Time: 867.1253s
Epoch 5 running
[Train #6] Loss: 0.0362 Acc: 88.5765% Time: 1022.0216s
[Test #6] Loss: 0.0383 Acc: 85.5016% Time: 1037.7133s
Epoch 6 running
[Train #7] Loss: 0.0360 Acc: 88.4379% Time: 1193.1716s
[Test #7] Loss: 0.0370 Acc: 87.8527% Time: 1208.9509s
Epoch 7 running
[Train #8] Loss: 0.0361 Acc: 88.2004% Time: 1365.9435s
[Test #8] Loss: 0.0369 Acc: 87.4608% Time: 1381.7345s
Epoch 8 ru

In [21]:
0 / 0

ZeroDivisionError: division by zero

In [23]:
model_path = dotenv['MODEL_DIR'] / '24-02-24_02_resNet.pt'

torch.save(model, model_path)

In [ ]:
import plotly.express as px
import numpy as np

In [ ]:
model.eval()
val_labels = []
val_predictions = []

with torch.no_grad():
    running_loss = 0.
    running_corrects = 0
    for inputs, labels in available_datasets['val']['dataloader']:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        val_labels += labels.tolist()
        val_predictions += preds.tolist()

In [ ]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(len(val_labels)):
    # Positive == 1, Negative == 0
    if val_labels[i] == 1 and val_predictions[i] == 1:
        TP += 1
    elif val_labels[i] == 0 and val_predictions[i] == 0:
        TN += 1
    elif val_labels[i] == 0 and val_predictions[i] == 1:
        FP += 1
    else:
        FN += 1

recall = TP / (TP + FN)
precision = TP / (TP + FP)
correct_classified = TP + TN
accuracy = correct_classified / len(val_predictions)

In [ ]:
TP, TN, FP, FN

In [ ]:
accuracy, recall, precision

In [ ]:
TP, TN, FP, FN

In [ ]:
accuracy, recall, precision

In [ ]:
fig = px.line(
    x=np.arange(1,num_epochs+1),
    y=train_accuary
)

# Only thing I figured is - I could do this 
fig.add_scatter(
    x=np.arange(1,num_epochs+1),
    y=test_accuary
) # Not what is desired - need a line 

# Show plot 
fig.show()